## Calculating Population Accessibility values and ratios

Calculates Population * Accessibility for each development scenario, service and mode. Also gives an accessibility ratio for each mesh block.

In [1]:
import pandas as pd
import numpy as np

from arcgis import GeoAccessor

Import the Population and Accessibility data

In [2]:
pop_acc_df_orig = GeoAccessor.from_featureclass("PopAcc_DS21")

In [3]:
#create a copy to do operations on - this didn't work??
pop_acc_df = pop_acc_df_orig
pop_acc_df

,OBJECTID,MB_CODE21,Person,Population_density,Cycle_GC_Actual_Access,Cycle_GC_EqualDev_Access,Cycle_GC_Greenfield_Access,Cycle_GC_mdINIS_Access,Cycle_GC_TODInfill_Access,Cycle_LC_Actual_Access,Cycle_LC_EqualDev_Access,Cycle_LC_Greenfield_Access,Cycle_LC_mdINIS_Access,Cycle_LC_TODInfill_Access,Cycle_Schools_Actual_Access,Cycle_Schools_EqualDev_Access,Cycle_Schools_Greenfield_Access,Cycle_Schools_mdINIS_Access,Cycle_Schools_TODInfill_Access,PT_GC_Actual_Access,PT_GC_EqualDev_Access,PT_GC_Greenfield_Access,PT_GC_mdINIS_Access,PT_GC_TODInfill_Access,PT_LC_Actual_Access,PT_LC_EqualDev_Access,PT_LC_Greenfield_Access,PT_LC_mdINIS_Access,PT_LC_TODInfill_Access,PT_Schools_Actual_Access,PT_Schools_EqualDev_Access,PT_Schools_Greenfield_Access,PT_Schools_mdINIS_Access,PT_Schools_TODInfill_Access,Walk_GC_Actual_Access,Walk_GC_EqualDev_Access,Walk_GC_Greenfield_Access,Walk_GC_mdINIS_Access,Walk_GC_TODInfill_Access,Walk_LC_Actual_Access,...,Walk_LC_Greenfield_Access,Walk_LC_mdINIS_Access,Walk_LC_TODInfill_Access,Walk_Schools_Actual_Access,Walk_Schools_EqualDev_Access,Walk_Schools_Greenfield_Access,Walk_Schools_mdINIS_Access,Walk_Schools_TODInfill_Access,Cycle_GC_Access,Cycle_LC_Access,Cycle_Schools_Access,Cycle_TC_Access,Cycle_empl_Access,PT_GC_Access,PT_LC_Access,PT_Schools_Access,PT_TC_Access,PT_empl_Access,Walk_GC_Access,Walk_LC_Access,Walk_Schools_Access,Walk_TC_Access,Walk_empl_Access,person_21,population_density_21,mb_code16,population_density_16,pop16_est,pop_percent_change,pop_change,pop_density_change,ds_equal_dev_pop_dens,ds_equal_dev_pop,ds_md_inis_dev_pop_dens,ds_md_inis_dev_pop,ds_tod_infill_pop_dens,ds_tod_infill_pop,ds_greenfield_pop_dens,ds_greenfield_pop,SHAPE
0,1,10636200000,105,2991.452991,0.998223,0.998223,0.998223,0.998936,0.998260,1.918131,1.785453,1.333934,1.334836,1.835951,5.211176,5.783418,4.592555,4.607091,4.993043,0.182282,0.182282,0.182282,0.182284,0.182283,1.122617,1.016298,0.210439,0.210451,0.369539,1.600937,1.832319,1.581862,1.582389,1.624227,0.178995,0.178995,0.178995,0.178995,0.178995,1.061917,...,0.208277,0.208277,0.365218,1.562926,1.805358,1.543373,1.543373,1.584446,0.998040,1.333934,4.592555,0.805379,9627.410537,0.181953,0.210439,1.581862,0.180864,1837.587594,0.178746,0.208277,1.543373,1.779153e-01,1810.650855,105.0,2991.452991,10636200000,2421.652422,85.0,23.5,20.0,569.0,2787.652422,97.0,2421.0,85.0,2421.0,85.0,2421.0,85.0,"{""rings"": [[[149.21605164087055, -35.357119318..."
1,2,10636210000,122,2541.666667,1.072899,1.072899,1.072899,1.074156,1.072975,1.982997,1.828655,1.391537,1.393076,1.949945,5.361751,5.866793,4.848402,4.873082,5.312768,0.271061,0.271061,0.271061,0.271062,0.271062,0.977402,1.098533,0.280052,0.280063,0.737542,1.953577,2.072464,1.948012,1.948540,2.066809,0.307273,0.307273,0.307273,0.307273,0.307273,1.048405,...,0.318301,0.318301,0.803831,2.026262,2.150123,2.020219,2.020219,2.156725,1.072767,1.391537,4.848402,0.912511,11137.352109,0.270632,0.280052,1.948012,0.270042,2644.710252,0.306917,0.318301,2.020219,3.067034e-01,2751.668754,122.0,2541.666667,10636210000,2437.500000,116.0,4.3,6.0,125.0,2803.500000,134.0,2437.0,116.0,2437.0,116.0,2437.0,116.0,"{""rings"": [[[149.21687245120268, -35.355530258..."
2,3,10636220000,136,2839.248434,0.977449,0.977449,0.977449,0.978732,0.977527,1.911184,1.741602,1.260963,1.262533,1.875576,4.937069,5.347028,4.559510,4.584680,5.075894,0.286684,0.286684,0.286684,0.286686,0.286685,1.039740,0.692534,0.288624,0.288637,1.070592,1.861949,1.878522,1.861311,1.861839,2.236127,0.283587,0.283587,0.283587,0.283587,0.283587,1.034004,...,0.285304,0.285304,1.063316,1.864730,1.883781,1.864301,1.864301,2.246039,0.977613,1.260963,4.559510,0.870032,11026.655835,0.287125,0.288624,1.861311,0.286618,3089.789840,0.283926,0.285304,1.864301,2.839205e-01,3109.134473,136.0,2839.248434,10636220000,2776.617954,133.0,2.3,3.0,62.0,3142.617954,150.0,2776.0,133.0,2776.0,133.0,2776.0,133.0,"{""rings"": [[[149.21892156188062, -35.347781808..."
3,4,10636230000,71,2171.253823,0.976798,0.976798,0.9

The accessibility and population columns have different notation for their development  (e.g. mdINIS vs md_inis_dev), so we create a dictionary giving the population development scenario column name for each of the accessibility names.

In [4]:
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

In [5]:
DS_pop_columns_dict = {"2016": "pop16_est",
                       "Actual": "person_21", 
                       "EqualDev": "ds_equal_dev_pop", 
                       "Greenfield": 'ds_greenfield_pop', 
                       "mdINIS": 'ds_md_inis_dev_pop', 
                       "TODInfill": "ds_tod_infill_pop"}

In [6]:
#calculate population accessibility
for column in pop_acc_df:
    
    # for every access column, multiply its access value with its respective population
    
    ###
    
    if column.split("_")[-1] == "Access":
        DS = column.split("_")[-2]
        
        
        #rename the 2016 columns to give a development scenario
        
        ##
        
        if DS not in DS_list:
            
            Access_2016_col = "_".join(column.split("_")[:-1]) + "_2016_" + column.split("_")[-1]
            
            pop_acc_df.rename(columns = {column : Access_2016_col}, inplace = True)
            
            column =  Access_2016_col
            DS = "2016"
            
            
        ##
        
        
        
        
        #calculate population accessibility
        
        ##
        
        #find the respective population column
        pop_col = DS_pop_columns_dict[DS]
        
        #multiply the two columns together to get a population accessibility series
        PA_ser = pop_acc_df[column].multiply(pop_acc_df[pop_col])
        
        
        # create a name for this new column
        PA_name = "PA_" + DS + "_" + "_".join(column.split("_")[:2])
        
        # add it to the overall dataframe
        pop_acc_df[PA_name] = PA_ser
        
        ##
        
    ###

In [7]:
# calculate population accessibility of employment and town centres (TC)
#     (these do not have accessibility columns, and change proportional to population change)

###


# go through each development scenario and divide the DS population by the 2016 populaiton

base_pop_col =  DS_pop_columns_dict["2016"]

for mode in ["Cycle", "PT", "Walk"]:
    
    #get the base accessibility data for empl and TC
    base_empl_Acc_col = mode + "_empl_2016_Access"
    base_TC_Acc_col = mode + "_TC_2016_Access"

    for DS in DS_list[:]:   

        #find the respective population column
        DS_pop_col = DS_pop_columns_dict[DS]
        
        #calculate population-accessibility
        
        ##
        
        # for employment
        alpha_empl = 1.134
        
        PA_ser_empl = alpha_empl* pop_acc_df[base_empl_Acc_col].multiply(pop_acc_df[DS_pop_col])
        
        # for town centres
        PA_ser_TC = pop_acc_df[base_TC_Acc_col].multiply(pop_acc_df[DS_pop_col])
        
        ##
       
        # create a name for the new employment and town centre columns
        PA_name_empl = "PA_" + DS + "_" + mode + "_empl"
        PA_name_TC = "PA_" + DS + "_" + mode + "_TC"

        # add it to the overall dataframe
        pop_acc_df[PA_name_empl] = PA_ser_empl
        pop_acc_df[PA_name_TC] = PA_ser_TC
###

In [8]:
pop_acc_df

,OBJECTID,MB_CODE21,Person,Population_density,Cycle_GC_Actual_Access,Cycle_GC_EqualDev_Access,Cycle_GC_Greenfield_Access,Cycle_GC_mdINIS_Access,Cycle_GC_TODInfill_Access,Cycle_LC_Actual_Access,Cycle_LC_EqualDev_Access,Cycle_LC_Greenfield_Access,Cycle_LC_mdINIS_Access,Cycle_LC_TODInfill_Access,Cycle_Schools_Actual_Access,Cycle_Schools_EqualDev_Access,Cycle_Schools_Greenfield_Access,Cycle_Schools_mdINIS_Access,Cycle_Schools_TODInfill_Access,PT_GC_Actual_Access,PT_GC_EqualDev_Access,PT_GC_Greenfield_Access,PT_GC_mdINIS_Access,PT_GC_TODInfill_Access,PT_LC_Actual_Access,PT_LC_EqualDev_Access,PT_LC_Greenfield_Access,PT_LC_mdINIS_Access,PT_LC_TODInfill_Access,PT_Schools_Actual_Access,PT_Schools_EqualDev_Access,PT_Schools_Greenfield_Access,PT_Schools_mdINIS_Access,PT_Schools_TODInfill_Access,Walk_GC_Actual_Access,Walk_GC_EqualDev_Access,Walk_GC_Greenfield_Access,Walk_GC_mdINIS_Access,Walk_GC_TODInfill_Access,Walk_LC_Actual_Access,...,PA_2016_PT_GC,PA_2016_PT_LC,PA_2016_PT_Schools,PA_2016_PT_TC,PA_2016_PT_empl,PA_2016_Walk_GC,PA_2016_Walk_LC,PA_2016_Walk_Schools,PA_2016_Walk_TC,PA_2016_Walk_empl,PA_Actual_Cycle_empl,PA_Actual_Cycle_TC,PA_EqualDev_Cycle_empl,PA_EqualDev_Cycle_TC,PA_Greenfield_Cycle_empl,PA_Greenfield_Cycle_TC,PA_mdINIS_Cycle_empl,PA_mdINIS_Cycle_TC,PA_TODInfill_Cycle_empl,PA_TODInfill_Cycle_TC,PA_Actual_PT_empl,PA_Actual_PT_TC,PA_EqualDev_PT_empl,PA_EqualDev_PT_TC,PA_Greenfield_PT_empl,PA_Greenfield_PT_TC,PA_mdINIS_PT_empl,PA_mdINIS_PT_TC,PA_TODInfill_PT_empl,PA_TODInfill_PT_TC,PA_Actual_Walk_empl,PA_Actual_Walk_TC,PA_EqualDev_Walk_empl,PA_EqualDev_Walk_TC,PA_Greenfield_Walk_empl,PA_Greenfield_Walk_TC,PA_mdINIS_Walk_empl,PA_mdINIS_Walk_TC,PA_TODInfill_Walk_empl,PA_TODInfill_Walk_TC
0,1,10636200000,105,2991.452991,0.998223,0.998223,0.998223,0.998936,0.998260,1.918131,1.785453,1.333934,1.334836,1.835951,5.211176,5.783418,4.592555,4.607091,4.993043,0.182282,0.182282,0.182282,0.182284,0.182283,1.122617,1.016298,0.210439,0.210451,0.369539,1.600937,1.832319,1.581862,1.582389,1.624227,0.178995,0.178995,0.178995,0.178995,0.178995,1.061917,...,15.466043,17.887311,134.458288,15.373467,1.771251e+05,15.193418,17.703518,131.186703,15.122801,1.745286e+05,1.146336e+06,84.564773,1.058996e+06,78.121742,9.279861e+05,68.457197,9.279861e+05,68.457197,9.279861e+05,68.457197,2.188016e+05,18.990753,2.021310e+05,17.543838,1.771251e+05,15.373467,1.771251e+05,15.373467,1.771251e+05,15.373467,2.155942e+05,1.868111e+01,1.991680e+05,17.257784,1.745286e+05,1.512280e+01,1.745286e+05,15.122801,1.745286e+05,15.122801
1,2,10636210000,122,2541.666667,1.072899,1.072899,1.072899,1.074156,1.072975,1.982997,1.828655,1.391537,1.393076,1.949945,5.361751,5.866793,4.848402,4.873082,5.312768,0.271061,0.271061,0.271061,0.271062,0.271062,0.977402,1.098533,0.280052,0.280063,0.737542,1.953577,2.072464,1.948012,1.948540,2.066809,0.307273,0.307273,0.307273,0.307273,0.307273,1.048405,...,31.393267,32.485994,225.969448,31.324871,3.478958e+05,35.602410,36.922903,234.345411,35.577600,3.619655e+05,1.540830e+06,111.326331,1.692387e+06,122.276462,1.465052e+06,105.851266,1.465052e+06,105.851266,1.465052e+06,105.851266,3.658904e+05,32.945123,4.018796e+05,36.185627,3.478958e+05,31.324871,3.478958e+05,31.324871,3.478958e+05,31.324871,3.806879e+05,3.741782e+01,4.181326e+05,41.098262,3.619655e+05,3.557760e+01,3.619655e+05,35.577600,3.619655e+05,35.577600
2,3,10636220000,136,2839.248434,0.977449,0.977449,0.977449,0.978732,0.977527,1.911184,1.741602,1.260963,1.262533,1.875576,4.937069,5.347028,4.559510,4.584680,5.075894,0.286684,0.286684,0.286684,0.286686,0.286685,1.039740,0.692534,0.288624,0.288637,1.070592,1.861949,1.878522,1.861311,1.861839,2.236127,0.283587,0.283587,0.283587,0.283587,0.283587,1.034004,...,38.187660,38.386962,247.554354,38.120142,4.660083e+05,37.762183,37.945456,247.952077,37.761424,4.689259e+05,1.700575e+06,118.324357,1.875634e+06,130.504806,1.663062e+06,115.714261,1.663062e+06,115.714261,1.663062e+06,115.714261,4.765197e+05,38.979995,5.255733e+05,42.992642,4.660

#### Estimated mode share increase calculations

In [57]:
pop_acc_df["PA_EqualDev_Cycle_Schools"].sum()/pop_acc_df["PA_2016_Cycle_Schools"].sum()

1.2170083634491946

In [10]:
#find the estimated mode share increase 
Mode_increase_list = []
mode_service_list = [] # contain the label for each mode and service
for mode in ["PT", "Cycle", "Walk"]:
    for service in ["Schools", "LC", "GC", "TC", "empl"]:
        mode_service_list.append(mode + "_" + service)
        
        DS_DeltaM_list = []
        for DS in DS_list:
            Base_PA = "_".join(["PA", "2016", mode, service])
            Scenario_PA = "_".join(["PA", DS, mode, service])
            try:
                mode_increase = pop_acc_df[Scenario_PA].sum()/pop_acc_df[Base_PA].sum()
            except:
                print("data not available: ", DS, mode, service)
                DS_DeltaM_list.append("N/A")
                continue
                
            DS_DeltaM_list.append(mode_increase)
        Mode_increase_list.append(DS_DeltaM_list)
        
    
    
MI_df = pd.DataFrame(Mode_increase_list, columns = DS_list)
MI_df.insert(loc = 0, column = "Mode-Service", value = mode_service_list)
# cols = DS_list
# cols[0] = "Mode-Service"
# MI_df = df
MI_df

,Mode-Service,2016,Actual,EqualDev,Greenfield,mdINIS,TODInfill
0,PT_Schools,1.0,1.257122,1.219063,1.178557,1.315114,1.283510
1,PT_LC,1.0,1.222725,1.194311,1.166935,1.230599,1.226590
2,PT_GC,1.0,1.256126,1.203745,1.177976,1.234009,1.262183
3,PT_TC,1.0,1.198131,1.108111,1.038812,1.109210,1.251056
4,PT_empl,1.0,1.229838,1.108152,1.023727,1.282922,1.290310
5,Cycle_Schools,1.0,1.233045,1.217008,1.190410,1.282787,1.264674
6,Cycle_LC,1.0,1.196716,1.186466,1.156135,1.191017,1.196754
7,Cycle_GC,1.0,1.205703,1.199108,1.160144,1.209208,1.214536
8,Cycle_TC,1.0,1.152424,1.126284,1.074406,1.125113,1.191255
9,Cycle_empl,1.0,1.174444,1.128246,1.034752,1.357388,1.239602


In [11]:
# MI_gk = MI_df.groupby("Mode")
# MI_gk.sum()

Combining each mode's increases

In [12]:
# incoropate weightings to give an overall mode_share increase

# define weightings
weights_dict = {"empl": 0.3441, "LC": 0.2901, "Schools": 0.2061, "GC":0.1191, "TC":0.0396}

#intialise arrays to put the overall mode share increase for each DS in
overall_MI_dict = {"PT" : [], "Cycle" : [], "Walk": []}
for mode in overall_MI_dict.keys():
    
    #intialise the correct amount of 0's
    overall_MI_dict[mode] += [0] * (len(MI_df.columns)-1)
    
    #convert to array
    overall_MI_dict[mode] = np.array(overall_MI_dict[mode])

    




# go through each service type and add according to its weight
for index, row in MI_df.iterrows():
    
    service = row["Mode-Service"].split("_")[-1]
    
    for mode in overall_MI_dict.keys():
        
        if row["Mode-Service"].split("_")[0] == mode:
            
            #add this rows data to the correct list with the correct weight
            weighted_MI = np.array([weights_dict[service] * MI  for MI in row[1:]])

            overall_MI_dict[mode] = overall_MI_dict[mode] + weighted_MI

            
            
overall_MI_table = []
for mode in overall_MI_dict.keys():
    overall_MI_table.append(overall_MI_dict[mode])
    
overall_MI_df = pd.DataFrame(overall_MI_table, columns = DS_list)
overall_MI_df.insert(loc = 0, column = "Mode", value = [mode for mode in overall_MI_dict.keys()])
overall_MI_df

,Mode,2016,Actual,EqualDev,Greenfield,mdINIS,TODInfill
0,PT,0.999,1.234043,1.166281,1.115127,1.260390,1.264229
1,Cycle,0.999,1.194659,1.170663,1.117516,1.265545,1.226199
2,Walk,0.999,1.252635,1.187599,1.121358,1.269897,1.301786


#### Emissions and economic differences

In [13]:
emissions_pkm = {"PT": -0.164, "Walk": -0.181, "Cycle": -0.181}
income_pkm = {"PT": 0.17, "Walk": 0.69, "Cycle":0.44}

km_by_mode = {"PT": 709100, "Walk": 226100, "Cycle": 187400}

In [14]:
def find_delta(overall_MI_df, impact_pkm, km_by_mode = km_by_mode):
    '''Finds DeltaE and DeltaI from overall accessibility measures
    '''
    
    impact_data_mode_table = []
    
    for index, row in overall_MI_df.iterrows():
        mode = row["Mode"]

        impact_data_mode = [(MI-1) * km_by_mode[mode]* impact_pkm[mode] for MI in row.values[1:]]

        impact_data_mode_table.append(impact_data_mode)
        
    impact_df = pd.DataFrame(impact_data_mode_table, columns = DS_list)
    impact_df.insert(loc = 0, column = "Mode", value = [mode for mode in overall_MI_dict.keys()])
    
    return impact_df

In [15]:
find_delta(overall_MI_df, emissions_pkm, km_by_mode = km_by_mode)#.to_csv(r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\Results\Emissions_effects_by_service.csv")

,Mode,2016,Actual,EqualDev,Greenfield,mdINIS,TODInfill
0,PT,116.2924,-27217.436539,-19337.235891,-13388.378805,-30281.424837,-30727.769512
1,Cycle,33.9194,-6602.721383,-5788.790184,-3986.076147,-9007.114427,-7672.545015
2,Walk,40.9241,-10338.857819,-7677.303169,-4966.476416,-11045.302032,-12350.302454


### Calculate a mode share increase by mesh block
This gives a mode share increase map

In [16]:
## calculate mode share increase columns

#each population accessibility for each scenario divided by the 2016 PA value
mode_list = ["PT", "Cycle", "Walk"]
service_list = ["empl", "LC", "GC", "TC", "Schools"]
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

for mode in mode_list:
    for service in service_list:
        for DS in DS_list[1:]: #exclude base population scenario
            
            Base_PA = "_".join(["PA", "2016", mode, service])
            Scenario_PA = "_".join(["PA", DS, mode, service])
            
            #find the mode increase for each scenario through the relative population-accessibility
            mode_increase_ser = pop_acc_df[Scenario_PA]/pop_acc_df[Base_PA]
            
            #replace infinite values with 1's
            for index, value in mode_increase_ser.items():
                if np.isinf(value):
                    mode_increase_ser[index] = 1
                    
            
            #checking for divide by zero errors - deal with these later
#             for index, value in pop_acc_df[Base_PA].items():
#                 print(index)
#                 if np.isnan(value):
#                     print("nan")
#                     print(pop_acc_df)
#                 elif int(value) == 0:
#                     print("inf")
            
            MI_col_name = "_".join(["MI", DS, mode, service])
            
            #add to pop_acc_ddf
            pop_acc_df[MI_col_name] = mode_increase_ser

### Calculate a combined accessibility measure for each mode and development scenario
Aim to export to a map for combined for each scenario.


Combined accessibility measures do not make sense, due to the different scales associated with each, the standardization process here will introduce significant error. - maybe should use median instead of maximum to more reasonable scales? Or set area = 1?

In [17]:
[print(col) for col in pop_acc_df.columns]

OBJECTID
MB_CODE21
Person
Population_density
Cycle_GC_Actual_Access
Cycle_GC_EqualDev_Access
Cycle_GC_Greenfield_Access
Cycle_GC_mdINIS_Access
Cycle_GC_TODInfill_Access
Cycle_LC_Actual_Access
Cycle_LC_EqualDev_Access
Cycle_LC_Greenfield_Access
Cycle_LC_mdINIS_Access
Cycle_LC_TODInfill_Access
Cycle_Schools_Actual_Access
Cycle_Schools_EqualDev_Access
Cycle_Schools_Greenfield_Access
Cycle_Schools_mdINIS_Access
Cycle_Schools_TODInfill_Access
PT_GC_Actual_Access
PT_GC_EqualDev_Access
PT_GC_Greenfield_Access
PT_GC_mdINIS_Access
PT_GC_TODInfill_Access
PT_LC_Actual_Access
PT_LC_EqualDev_Access
PT_LC_Greenfield_Access
PT_LC_mdINIS_Access
PT_LC_TODInfill_Access
PT_Schools_Actual_Access
PT_Schools_EqualDev_Access
PT_Schools_Greenfield_Access
PT_Schools_mdINIS_Access
PT_Schools_TODInfill_Access
Walk_GC_Actual_Access
Walk_GC_EqualDev_Access
Walk_GC_Greenfield_Access
Walk_GC_mdINIS_Access
Walk_GC_TODInfill_Access
Walk_LC_Actual_Access
Walk_LC_EqualDev_Access
Walk_LC_Greenfield_Access
Walk_LC_mdINIS_

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [18]:
#calculate combined population columns

# define weightings
weights_dict = {"empl": 0.3441, "LC": 0.2901, "Schools": 0.2061, "GC":0.1191, "TC":0.0396}

for mode in ["PT", "Cycle", "Walk"]:
    for DS in DS_list:
        mode_DS_combined_data = [0]*len(pop_acc_df)
        for service in weights_dict.keys():
            acc_col = "_".join([mode, service, DS, "Access"])
            
            if (service == "empl") or (service == "TC"):
                acc_col = "_".join([mode, service, "2016", "Access"])
            
            
            service_data = pop_acc_df[acc_col]
            
            #normalise the service accessibility data
            N = max(service_data)
            service_data = service_data/N

#             # use for population accessibility version
#             if (service == "empl") and (DS != "2016"):
#                 service_data = 1.134 * service_data
            
            #apply weighting to the series
            weighted_service_data =  weights_dict[service]*service_data
            
            # add the weightings to the current total
            for i in range(0, len(service_data)):
                
                mode_DS_combined_data[i] += weighted_service_data[i]

                
                
        #add to pop_acc_df dataframe
        
        #create a column name
        mode_DS_combined_name = mode + "_" + DS + "_Combined_Accessibility"
        
        pop_acc_df[mode_DS_combined_name] = mode_DS_combined_data
        #print(pop_acc_df[mode_DS_combined_name])

# for column in pop_acc_df:
    
#     # for every access column,
    
#     ###
    
#     if column.split("_")[-1] == "Access":
#         DS = column.split("_")[-2]
#         mode = column.split("_")[0]
#         service = column.split("_")[1]
        
        

In [19]:
# calculate combined values for population-accessibility and mode-share increase
mode_list = ["PT", "Cycle", "Walk"]
service_list = ["empl", "LC", "GC", "TC", "Schools"]
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

# define weightings
weights_dict = {"empl": 0.3441, "LC": 0.2901, "Schools": 0.2061, "GC":0.1191, "TC":0.0396}

for mode in mode_list:
    for DS in DS_list:
        mode_DS_combined_PA_data = [0]*len(pop_acc_df)
        mode_DS_combined_MI_data = [0]*len(pop_acc_df)
        for service in service_list:
            
            
            PA_col =  "_".join(["PA", DS, mode, service])
            MI_col =  "_".join(["MI", DS, mode, service])
            
            PA_data = pop_acc_df[PA_col]
            
            
            # no 2016 data for MI_col
            if DS != "2016":
                MI_data = pop_acc_df[MI_col]
            
            #fill with 1's if thats the case (column aribratrily chosen here)
            else:
                MI_data = pop_acc_df["PT_Schools_Actual_Access"]/pop_acc_df["PT_Schools_Actual_Access"]
                
            
            #normalise the population accessibility data (it is a relative measure due to accessibility values)
            N = max(PA_data)
            PA_data = PA_data/N
            
            
            #apply weighting to the series
            weighted_PA_data = weights_dict[service]*PA_data
            weighted_MI_data = weights_dict[service]*MI_data
            
            # add the weightings to the current total
            for i in range(0, len(pop_acc_df)):
                
                mode_DS_combined_PA_data[i] += weighted_PA_data[i]
                mode_DS_combined_MI_data[i] += weighted_MI_data[i]
            
            
        #add to pop_acc_df dataframe
        
        #create a column name
        PA_combined_col_name =  "_".join(["PA", DS, mode, "Combined"])
        MI_combined_col_name =  "_".join(["MI", DS, mode, "Combined"])
        
        pop_acc_df[PA_combined_col_name] = mode_DS_combined_PA_data
        pop_acc_df[MI_combined_col_name] = mode_DS_combined_MI_data


In [20]:
pop_acc_df#["SHAPE"]#["PT_Schools_Actual_Access"]/pop_acc_df["PT_Schools_Actual_Access"]

,OBJECTID,MB_CODE21,Person,Population_density,Cycle_GC_Actual_Access,Cycle_GC_EqualDev_Access,Cycle_GC_Greenfield_Access,Cycle_GC_mdINIS_Access,Cycle_GC_TODInfill_Access,Cycle_LC_Actual_Access,Cycle_LC_EqualDev_Access,Cycle_LC_Greenfield_Access,Cycle_LC_mdINIS_Access,Cycle_LC_TODInfill_Access,Cycle_Schools_Actual_Access,Cycle_Schools_EqualDev_Access,Cycle_Schools_Greenfield_Access,Cycle_Schools_mdINIS_Access,Cycle_Schools_TODInfill_Access,PT_GC_Actual_Access,PT_GC_EqualDev_Access,PT_GC_Greenfield_Access,PT_GC_mdINIS_Access,PT_GC_TODInfill_Access,PT_LC_Actual_Access,PT_LC_EqualDev_Access,PT_LC_Greenfield_Access,PT_LC_mdINIS_Access,PT_LC_TODInfill_Access,PT_Schools_Actual_Access,PT_Schools_EqualDev_Access,PT_Schools_Greenfield_Access,PT_Schools_mdINIS_Access,PT_Schools_TODInfill_Access,Walk_GC_Actual_Access,Walk_GC_EqualDev_Access,Walk_GC_Greenfield_Access,Walk_GC_mdINIS_Access,Walk_GC_TODInfill_Access,Walk_LC_Actual_Access,...,Walk_EqualDev_Combined_Accessibility,Walk_Greenfield_Combined_Accessibility,Walk_mdINIS_Combined_Accessibility,Walk_TODInfill_Combined_Accessibility,PA_2016_PT_Combined,MI_2016_PT_Combined,PA_Actual_PT_Combined,MI_Actual_PT_Combined,PA_EqualDev_PT_Combined,MI_EqualDev_PT_Combined,PA_Greenfield_PT_Combined,MI_Greenfield_PT_Combined,PA_mdINIS_PT_Combined,MI_mdINIS_PT_Combined,PA_TODInfill_PT_Combined,MI_TODInfill_PT_Combined,PA_2016_Cycle_Combined,MI_2016_Cycle_Combined,PA_Actual_Cycle_Combined,MI_Actual_Cycle_Combined,PA_EqualDev_Cycle_Combined,MI_EqualDev_Cycle_Combined,PA_Greenfield_Cycle_Combined,MI_Greenfield_Cycle_Combined,PA_mdINIS_Cycle_Combined,MI_mdINIS_Cycle_Combined,PA_TODInfill_Cycle_Combined,MI_TODInfill_Cycle_Combined,PA_2016_Walk_Combined,MI_2016_Walk_Combined,PA_Actual_Walk_Combined,MI_Actual_Walk_Combined,PA_EqualDev_Walk_Combined,MI_EqualDev_Walk_Combined,PA_Greenfield_Walk_Combined,MI_Greenfield_Walk_Combined,PA_mdINIS_Walk_Combined,MI_mdINIS_Walk_Combined,PA_TODInfill_Walk_Combined,MI_TODInfill_Walk_Combined
0,1,10636200000,105,2991.452991,0.998223,0.998223,0.998223,0.998936,0.998260,1.918131,1.785453,1.333934,1.334836,1.835951,5.211176,5.783418,4.592555,4.607091,4.993043,0.182282,0.182282,0.182282,0.182284,0.182283,1.122617,1.016298,0.210439,0.210451,0.369539,1.600937,1.832319,1.581862,1.582389,1.624227,0.178995,0.178995,0.178995,0.178995,0.178995,1.061917,...,0.274415,0.148513,0.138272,0.166193,0.014555,0.999,0.025585,2.790752,0.027686,2.445269,0.004029,0.999215,0.012859,0.999301,0.004257,1.224063,0.018786,0.999,0.019292,1.425323,0.019780,1.313105,0.007058,0.999022,0.017053,0.999955,0.006433,1.126176,0.020278,0.999,0.019579,2.706254,0.022781,2.458034,0.004613,0.999166,0.018043,0.999166,0.005140,1.223248
1,2,10636210000,122,2541.666667,1.072899,1.072899,1.072899,1.074156,1.072975,1.982997,1.828655,1.391537,1.393076,1.949945,5.361751,5.866793,4.848402,4.873082,5.312768,0.271061,0.271061,0.271061,0.271062,0.271062,0.977402,1.098533,0.280052,0.280063,0.737542,1.953577,2.072464,1.948012,1.948540,2.066809,0.307273,0.307273,0.307273,0.307273,0.307273,1.048405,...,0.333816,0.213818,0.200129,0.278612,0.025987,0.999,0.031097,1.811226,0.044064,2.148857,0.007453,0.999189,0.023110,0.999257,0.008601,1.485663,0.027479,0.999,0.023707,1.203321,0.028666,1.309311,0.010523,0.999015,0.024999,1.000524,0.009476,1.135176,0.038664,0.999,0.028058,1.751302,0.039002,2.022585,0.009248,0.999138,0.034669,0.999138,0.010817,1.455577
2,3,10636220000,136,2839.248434,0.977449,0.977449,0.977449,0.978732,0.977527,1.911184,1.741602,1.260963,1.262533,1.875576,4.937069,5.347028,4.559510,4.584680,5.075894,0.286684,0.286684,0.286684,0.286686,0.286685,1.039740,0.692534,0.288624,0.288637,1.070592,1.861949,1.878522,1.861311,1.861839,2.236127,0.283587,0.283587,0.283587,0.283587,0.283587,1.034004,...,0.259037,0.201545,0.188954,0.318368,0.030003,0.999,0.035807,1.793406,0.039832,1.586502,0.008969,0.998817,0.026818,0.998889,0.011855,1.826288,0.029320,0.999,0.025016,1.191930,0.030103,1.291528,0.011396,0.998980,0.026669,1.000635,0.01038

### Population Accessibility density

Population accessibility density is better layer for visualisation, as just using the population field means higher population areas also get higher PA values.

This will be calculated by dividing every PA field by its area value.

Create a new df here as the old one is getting large and cumbersome

In [21]:
area = pop_acc_df["Person"]/pop_acc_df["Population_density"]
area

0       0.0351
1       0.0480
2       0.0479
3       0.0327
4       0.0222
         ...  
7180       NaN
7181    0.0634
7182       NaN
7183    2.0587
7184    0.0422
Length: 7185, dtype: float64

In [22]:
pop_acc_dens_df = pd.DataFrame()

for column in pop_acc_df.columns:
    if column[:2] == "PA":
        
        pop_acc_dens = pop_acc_df[column]/area
        
        pop_acc_dens_col_name = "PA_dens" + column[2:]
        
        pop_acc_dens_df[pop_acc_dens_col_name] = pop_acc_dens

# add geospatial data
pop_acc_dens_df["SHAPE"] = pop_acc_df["SHAPE"]


pop_acc_dens_df

,PA_dens_Actual_Cycle_GC,PA_dens_EqualDev_Cycle_GC,PA_dens_Greenfield_Cycle_GC,PA_dens_mdINIS_Cycle_GC,PA_dens_TODInfill_Cycle_GC,PA_dens_Actual_Cycle_LC,PA_dens_EqualDev_Cycle_LC,PA_dens_Greenfield_Cycle_LC,PA_dens_mdINIS_Cycle_LC,PA_dens_TODInfill_Cycle_LC,PA_dens_Actual_Cycle_Schools,PA_dens_EqualDev_Cycle_Schools,PA_dens_Greenfield_Cycle_Schools,PA_dens_mdINIS_Cycle_Schools,PA_dens_TODInfill_Cycle_Schools,PA_dens_Actual_PT_GC,PA_dens_EqualDev_PT_GC,PA_dens_Greenfield_PT_GC,PA_dens_mdINIS_PT_GC,PA_dens_TODInfill_PT_GC,PA_dens_Actual_PT_LC,PA_dens_EqualDev_PT_LC,PA_dens_Greenfield_PT_LC,PA_dens_mdINIS_PT_LC,PA_dens_TODInfill_PT_LC,PA_dens_Actual_PT_Schools,PA_dens_EqualDev_PT_Schools,PA_dens_Greenfield_PT_Schools,PA_dens_mdINIS_PT_Schools,PA_dens_TODInfill_PT_Schools,PA_dens_Actual_Walk_GC,PA_dens_EqualDev_Walk_GC,PA_dens_Greenfield_Walk_GC,PA_dens_mdINIS_Walk_GC,PA_dens_TODInfill_Walk_GC,PA_dens_Actual_Walk_LC,PA_dens_EqualDev_Walk_LC,PA_dens_Greenfield_Walk_LC,PA_dens_mdINIS_Walk_LC,PA_dens_TODInfill_Walk_LC,...,PA_dens_TODInfill_Cycle_TC,PA_dens_Actual_PT_empl,PA_dens_Actual_PT_TC,PA_dens_EqualDev_PT_empl,PA_dens_EqualDev_PT_TC,PA_dens_Greenfield_PT_empl,PA_dens_Greenfield_PT_TC,PA_dens_mdINIS_PT_empl,PA_dens_mdINIS_PT_TC,PA_dens_TODInfill_PT_empl,PA_dens_TODInfill_PT_TC,PA_dens_Actual_Walk_empl,PA_dens_Actual_Walk_TC,PA_dens_EqualDev_Walk_empl,PA_dens_EqualDev_Walk_TC,PA_dens_Greenfield_Walk_empl,PA_dens_Greenfield_Walk_TC,PA_dens_mdINIS_Walk_empl,PA_dens_mdINIS_Walk_TC,PA_dens_TODInfill_Walk_empl,PA_dens_TODInfill_Walk_TC,PA_dens_2016_PT_Combined,PA_dens_Actual_PT_Combined,PA_dens_EqualDev_PT_Combined,PA_dens_Greenfield_PT_Combined,PA_dens_mdINIS_PT_Combined,PA_dens_TODInfill_PT_Combined,PA_dens_2016_Cycle_Combined,PA_dens_Actual_Cycle_Combined,PA_dens_EqualDev_Cycle_Combined,PA_dens_Greenfield_Cycle_Combined,PA_dens_mdINIS_Cycle_Combined,PA_dens_TODInfill_Cycle_Combined,PA_dens_2016_Walk_Combined,PA_dens_Actual_Walk_Combined,PA_dens_EqualDev_Walk_Combined,PA_dens_Greenfield_Walk_Combined,PA_dens_mdINIS_Walk_Combined,PA_dens_TODInfill_Walk_Combined,SHAPE
0,2986.136715,2758.621537,2417.348769,2419.076984,2417.438986,5737.998872,4934.156627,3230.324733,3232.509437,4446.034922,15588.988105,15982.664877,11121.571564,11156.772630,12091.415586,545.287286,503.741587,441.423040,441.427596,441.426061,3358.255052,2808.573111,509.609996,509.638006,894.895760,4789.126555,5063.673665,3830.720430,3831.996450,3933.312156,535.455717,494.659091,433.464151,433.464151,433.464151,3176.674658,2797.336051,504.373722,504.373722,884.430405,...,1950.347494,6.233663e+06,541.047089,5.758717e+06,499.824453,5.046298e+06,437.990500,5.046298e+06,437.990500,5.046298e+06,437.990500,6.142285e+06,5.322253e+02,5.674301e+06,491.674770,4.972326e+06,4.308490e+02,4.972326e+06,430.849026,4.972326e+06,430.849026,0.414683,0.728913,0.788771,0.114798,0.366357,0.121280,0.535227,0.549625,0.563520,0.201072,0.485840,0.183290,0.577708,0.557793,0.649043,0.131433,0.514043,0.146452,"{""rings"": [[[149.21605164087055, -35.357119318..."
1,2726.952432,2995.177261,2592.840017,2595.875900,2593.022767,5040.118527,5104.993872,3362.881493,3366.601013,4712.366550,13627.784523,16378.129560,11716.972335,11776.613639,12839.188825,688.945545,756.710680,655.062976,655.067523,655.065991,2484.230729,3066.738498,676.791532,676.819484,1782.392524,4965.341563,5785.627805,4707.696858,4708.971562,4994.789501,780.984428,857.802568,742.575357,742.575357,742.575357,2664.695413,3150.662052,769.227150,769.227150,1942.591005,...,2205.234706,7.622716e+06,686.356723,8.372491e+06,753.867220,7.247828e+06,652.601474,7.247828e+06,652.601474,7.247828e+06,652.601474,7.930997e+06,7.795379e+02,8.711095e+06,856.213795,7.540948e+06,7.412000e+02,7.540948e+06,741.200002,7.540948e+06,741.200002,0.541396,0.647864,0.918008,0.155266,0.481468,0.179179,0.572486,0.493889,0.597215,0.219221,0.520807,0.197412,0.805491,0.584539,0.812551,0.192666,0.722263,0.225356,"{""rings"": [[[149.21687245120268, -35.355530258..."
2,2775

### Mode share increase by mesh block

Visualising mode share increase involves finding the correct way to disaggrate the overall mode share increase for each service, and for the combined outcome.

This will be done by finding the proportion of the mode share increase that each mesh block contributes too.

In [23]:
## mode share increase by service

#find the population accessibility sum for each service
#each population accessibility for each scenario divided by the 2016 PA value
mode_list = ["PT", "Cycle", "Walk"]
service_list = ["empl", "LC", "GC", "TC", "Schools"]
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

for mode in mode_list:
    for DS in DS_list[1:]: #exclude base population scenario
        for service in service_list:
            
            Base_PA = "_".join(["PA", "2016", mode, service])
            PA_col =  "_".join(["PA", DS, mode, service])
            
            # find contibution by each mesh block
            mode_increase_ser = pop_acc_df[PA_col]/pop_acc_df[Base_PA].sum()
            
            #divide through by area (due to population variable being used)
            area = pop_acc_df["Person"]/pop_acc_df["Population_density"]
            mode_increase_ser = mode_increase_ser/area
            
            #replace any nan values with0's
            mode_increase_ser = mode_increase_ser.replace(np.nan, 0)
            
            MI_col_name = "_".join(["MI_dens", DS, mode, service])
            
            #add to pop_acc_df (replaces the relative mode share increase value)
            pop_acc_df[MI_col_name] = mode_increase_ser
            
            #create a non-density column (used to sum to the correct total)
            MI_col_name =  "_".join(["MI", DS, mode, service])
            pop_acc_df[MI_col_name] = mode_increase_ser*area.fillna(0)


In [24]:
# combined mode share increase

#this must not only consider the weightings for each service, but the sum of each PA value
# calculate combined values for population-accessibility and mode-share increase
mode_list = ["PT", "Cycle", "Walk"]
service_list = ["empl", "LC", "GC", "TC", "Schools"]
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

# define weightings
weights_dict = {"empl": 0.3441, "LC": 0.2901, "Schools": 0.2061, "GC":0.1191, "TC":0.0396}

for mode in mode_list:
    for DS in DS_list[1:]: #exclude base population scenario
        
        #initialise combined data
        mode_DS_combined_MI_data = [0]*len(pop_acc_df)
        
        for service in service_list:
            
            PA_col =  "_".join(["PA", DS, mode, service])
            Base_PA = "_".join(["PA", "2016", mode, service])
            
            # get the modified weighitng value (each PA sum is specific for each service)
            wp_C = weights_dict[service]/pop_acc_df[Base_PA].sum()
            
            weighted_MI_data = wp_C*pop_acc_df[PA_col]
            
            #divide through by area (due to population variable being used)
            area = pop_acc_df["Person"]/pop_acc_df["Population_density"]
            weighted_MI_data = weighted_MI_data/area
            
            #replace any nan values with 0's
            weighted_MI_data = weighted_MI_data.replace(np.nan,0)
            
            
            # add the weightings to the current total
            for i in range(0, len(pop_acc_df)):
                mode_DS_combined_MI_data[i] += weighted_MI_data[i]
                
        
        #add to pop_acc_df dataframe (replaced existing MI column)
        
        #create a column name
        MI_combined_col_name =  "_".join(["MI_dens", DS, mode, "Combined"])
        
        pop_acc_df[MI_combined_col_name] = mode_DS_combined_MI_data
        
        #create a non-density column
        MI_combined_col_name =  "_".join(["MI", DS, mode, "Combined"])
        pop_acc_df[MI_combined_col_name] = mode_DS_combined_MI_data*area.fillna(0)
        
        


In [61]:
pop_acc_df["OBJECTID"]*0

0       0
1       0
2       0
3       0
4       0
       ..
7180    0
7181    0
7182    0
7183    0
7184    0
Name: OBJECTID, Length: 7185, dtype: int64

In [81]:
## DOESN'T WORK
#Note: Combined columns can't be directly added (PA Combined is meaningless), but must be summed from weighted services for any new measure

#getting just the mode increase (excluding the mode contribution already there)
new_df_cols = ["OBJECTID", "MB_CODE21", "Person", "Population_density"]

mode_list = ["PT", "Cycle", "Walk"]
service_list = ["empl", "LC", "GC", "TC", "Schools"]
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

# define weightings
weights_dict = {"empl": 0.3441, "LC": 0.2901, "Schools": 0.2061, "GC":0.1191, "TC":0.0396}

#create an area column
area = pop_acc_df["Person"]/pop_acc_df["Population_density"]

for mode in mode_list:
    for DS in DS_list[1:]: #exclude base population scenario
        for service in service_list:
            
            Base_PA = "_".join(["PA", "2016", mode, service])
            PA_col =  "_".join(["PA", DS, mode, service])
            
            if Base_PA not in new_df_cols:
                new_df_cols.append(Base_PA)
            new_df_cols.append(PA_col)
            
MI_MB_df = pop_acc_df[new_df_cols].copy()



#add the mode increase values
for mode in mode_list:
    for DS in DS_list[1:]: #exclude base population scenario
        
        #initialise a 0-valued series of the correct length
        combined_ser = pop_acc_df["OBJECTID"]*0
        
        for service in service_list:
            
            Base_PA = "_".join(["PA", "2016", mode, service])
            PA_col =  "_".join(["PA", DS, mode, service])
            
            # find contibution by each mesh block (TO THE INCREASE - subtract original) 
            no_nulls_base = pop_acc_df[Base_PA].replace(np.nan, 0) #required as greenfield base will have PA nulls (study site is smaller)
            mode_increase_ser = pop_acc_df[PA_col].subtract(no_nulls_base)/pop_acc_df[Base_PA].sum()
            
            #divide through by area (due to population variable being used)
            area = pop_acc_df["Person"]/pop_acc_df["Population_density"]
            #mode_increase_ser = mode_increase_ser/area
            
            #replace any nan values with0's
            mode_increase_ser = mode_increase_ser.replace(np.nan, 0)
            
            #replace any negative values with 0's
            for index, value in mode_increase_ser.iteritems():
                if value < 0:
                    mode_increase_ser[index] = 0
                    
            
            #removed MI density coloumn - I don't think this is meaningful 
            #MI_col_name = "_".join(["MI_dens", DS, mode, service])
            
            #add to pop_acc_df (replaces the relative mode share increase value)
            #MI_MB_df[MI_col_name] = mode_increase_ser
            
            #create a non-density column (used to sum to the correct total)
            MI_col_name =  "_".join(["MI", DS, mode, service])
            MI_MB_df[MI_col_name] = mode_increase_ser#*area.fillna(0)
            
            
            
            #add this mode increase to the combined column
            combined_ser += weights_dict[service] *  mode_increase_ser
            
        #create the combined column column
        MI_col_name =  "_".join(["MI", DS, mode, "Combined"])
        MI_MB_df[MI_col_name] = combined_ser

            
MI_MB_df["SHAPE"] = pop_acc_df["SHAPE"]
MI_MB_df

,OBJECTID,MB_CODE21,Person,Population_density,PA_2016_PT_empl,PA_Actual_PT_empl,PA_2016_PT_LC,PA_Actual_PT_LC,PA_2016_PT_GC,PA_Actual_PT_GC,PA_2016_PT_TC,PA_Actual_PT_TC,PA_2016_PT_Schools,PA_Actual_PT_Schools,PA_EqualDev_PT_empl,PA_EqualDev_PT_LC,PA_EqualDev_PT_GC,PA_EqualDev_PT_TC,PA_EqualDev_PT_Schools,PA_Greenfield_PT_empl,PA_Greenfield_PT_LC,PA_Greenfield_PT_GC,PA_Greenfield_PT_TC,PA_Greenfield_PT_Schools,PA_mdINIS_PT_empl,PA_mdINIS_PT_LC,PA_mdINIS_PT_GC,PA_mdINIS_PT_TC,PA_mdINIS_PT_Schools,PA_TODInfill_PT_empl,PA_TODInfill_PT_LC,PA_TODInfill_PT_GC,PA_TODInfill_PT_TC,PA_TODInfill_PT_Schools,PA_2016_Cycle_empl,PA_Actual_Cycle_empl,PA_2016_Cycle_LC,PA_Actual_Cycle_LC,PA_2016_Cycle_GC,PA_Actual_Cycle_GC,...,MI_mdINIS_Cycle_TC,MI_mdINIS_Cycle_Schools,MI_mdINIS_Cycle_Combined,MI_TODInfill_Cycle_empl,MI_TODInfill_Cycle_LC,MI_TODInfill_Cycle_GC,MI_TODInfill_Cycle_TC,MI_TODInfill_Cycle_Schools,MI_TODInfill_Cycle_Combined,MI_Actual_Walk_empl,MI_Actual_Walk_LC,MI_Actual_Walk_GC,MI_Actual_Walk_TC,MI_Actual_Walk_Schools,MI_Actual_Walk_Combined,MI_EqualDev_Walk_empl,MI_EqualDev_Walk_LC,MI_EqualDev_Walk_GC,MI_EqualDev_Walk_TC,MI_EqualDev_Walk_Schools,MI_EqualDev_Walk_Combined,MI_Greenfield_Walk_empl,MI_Greenfield_Walk_LC,MI_Greenfield_Walk_GC,MI_Greenfield_Walk_TC,MI_Greenfield_Walk_Schools,MI_Greenfield_Walk_Combined,MI_mdINIS_Walk_empl,MI_mdINIS_Walk_LC,MI_mdINIS_Walk_GC,MI_mdINIS_Walk_TC,MI_mdINIS_Walk_Schools,MI_mdINIS_Walk_Combined,MI_TODInfill_Walk_empl,MI_TODInfill_Walk_LC,MI_TODInfill_Walk_GC,MI_TODInfill_Walk_TC,MI_TODInfill_Walk_Schools,MI_TODInfill_Walk_Combined,SHAPE
0,1,10636200000,105,2991.452991,1.771251e+05,2.188016e+05,17.887311,117.874752,15.466043,19.139584,15.373467,18.990753,134.458288,168.098342,2.021310e+05,98.580916,17.681330,17.543838,177.734946,1.771251e+05,17.887311,15.493949,15.373467,134.458287,1.771251e+05,17.888294,15.494109,15.373467,134.503075,1.771251e+05,31.410841,15.494055,15.373467,138.059257,9.279861e+05,1.146336e+06,113.384398,201.403760,84.833436,104.813399,...,0.000000,4.296314e-07,1.302735e-07,0.000000,0.000049,3.367289e-08,0.000000,0.000012,0.000017,3.190951e-05,0.000229,0.000021,7.685797e-05,0.000046,0.000092,0.000019,0.000197,0.000013,4.611478e-05,0.000061,0.000080,0.000000,0.000000,1.241804e-07,0.000000e+00,0.000000e+00,1.478988e-08,0.000000,2.169049e-19,1.241804e-07,0.000000e+00,0.000000e+00,1.478988e-08,0.000000,3.257802e-05,1.241804e-07,0.000000e+00,0.000005,0.000010,"{""rings"": [[[149.21605164087055, -35.357119318..."
1,2,10636210000,122,2541.666667,3.478958e+05,3.658904e+05,32.485994,119.243075,31.393267,33.069386,31.324871,32.945123,225.969448,238.336395,4.018796e+05,147.203448,36.322113,36.185627,277.710135,3.478958e+05,32.485994,31.443023,31.324871,225.969449,3.478958e+05,32.487335,31.443241,31.324871,226.030635,3.478958e+05,85.554841,31.443168,31.324871,239.749896,1.465052e+06,1.540830e+06,161.418312,241.925689,124.440955,130.893717,...,0.000000,9.954542e-07,2.988234e-07,0.000000,0.000074,4.352952e-08,0.000000,0.000019,0.000025,1.454798e-05,0.000222,0.000011,3.974802e-05,0.000018,0.000076,0.000044,0.000279,0.000033,1.192441e-04,0.000075,0.000120,0.000000,0.000000,2.416771e-07,0.000000e+00,3.961473e-12,2.878456e-08,0.000000,2.464040e-18,2.416771e-07,0.000000e+00,3.999936e-12,2.878456e-08,0.000000,1.375445e-04,2.416771e-07,0.000000e+00,0.000022,0.000044,"{""rings"": [[[149.21687245120268, -35.355530258..."
2,3,10636220000,136,2839.248434,4.660083e+05,4.765197e+05,38.386962,141.404665,38.187660,38.988983,38.120142,38.979995,247.554354,253.225065,5.255733e+05,103.880103,43.002555,42.992642,281.778355,4.660083e+05,38.386962,38.128932,38.120142,247.554356,4.660083e+05,38.388713,38.129196,38.120142,247.624522,4.660083e+05,142.388703,38.129098,38.120142,297.404871,1.663062e+06,1.700575e+06,167.708108,259.921010,130.022539,132.933099,...,0.000000,1.164034e-06,3.409619e-07,0.000000,0.000093,0.000000e+00,0.000000,0.000024,0.000032,8.218947e-06,0.000251,0.000005,1.839770e-05,0.000008,0.000078

In [82]:
overall_MI_df

,Mode,2016,Actual,EqualDev,Greenfield,mdINIS,TODInfill
0,PT,0.999,1.234043,1.166281,1.115127,1.260390,1.264229
1,Cycle,0.999,1.194659,1.170663,1.117516,1.265545,1.226199
2,Walk,0.999,1.252635,1.187599,1.121358,1.269897,1.301786


In [83]:
MI_MB_df["MI_Greenfield_Walk_Combined"].sum()

0.12304436817327832

In [84]:
pop_acc_df["PA_EqualDev_Walk_Combined"].sum()/pop_acc_df["PA_2016_Walk_Combined"].sum()

0.8468821896866374

In [37]:
## DOESN'T WORK

# find PA2/PA1 for each mesh block
mode_list = ["PT", "Cycle", "Walk"]
service_list = ["empl", "LC", "GC", "TC", "Schools"]
DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

for mode in mode_list:
    for DS in DS_list[1:]: #exclude base population scenario
        service = "Combined"

        Base_PA = "_".join(["PA", "2016", mode, service])
        PA_col =  "_".join(["PA", DS, mode, service])

        MI_by_MB = pop_acc_df[PA_col]/pop_acc_df[Base_PA]

        MIMB_col_name = "_".join(["MI_MB", DS, mode, service])

        pop_acc_df[MIMB_col_name] = MI_by_MB

MI_by_MB[np.isnan(MI_by_MB)]

11     NaN
25     NaN
28     NaN
31     NaN
33     NaN
        ..
7178   NaN
7179   NaN
7180   NaN
7182   NaN
7183   NaN
Length: 2715, dtype: float64

In [38]:
pop_acc_df

,OBJECTID,MB_CODE21,Person,Population_density,Cycle_GC_Actual_Access,Cycle_GC_EqualDev_Access,Cycle_GC_Greenfield_Access,Cycle_GC_mdINIS_Access,Cycle_GC_TODInfill_Access,Cycle_LC_Actual_Access,Cycle_LC_EqualDev_Access,Cycle_LC_Greenfield_Access,Cycle_LC_mdINIS_Access,Cycle_LC_TODInfill_Access,Cycle_Schools_Actual_Access,Cycle_Schools_EqualDev_Access,Cycle_Schools_Greenfield_Access,Cycle_Schools_mdINIS_Access,Cycle_Schools_TODInfill_Access,PT_GC_Actual_Access,PT_GC_EqualDev_Access,PT_GC_Greenfield_Access,PT_GC_mdINIS_Access,PT_GC_TODInfill_Access,PT_LC_Actual_Access,PT_LC_EqualDev_Access,PT_LC_Greenfield_Access,PT_LC_mdINIS_Access,PT_LC_TODInfill_Access,PT_Schools_Actual_Access,PT_Schools_EqualDev_Access,PT_Schools_Greenfield_Access,PT_Schools_mdINIS_Access,PT_Schools_TODInfill_Access,Walk_GC_Actual_Access,Walk_GC_EqualDev_Access,Walk_GC_Greenfield_Access,Walk_GC_mdINIS_Access,Walk_GC_TODInfill_Access,Walk_LC_Actual_Access,...,MI_MB_Actual_Walk_empl,MI_MB_Actual_Walk_LC,MI_MB_Actual_Walk_GC,MI_MB_Actual_Walk_TC,MI_MB_Actual_Walk_Schools,MI_MB_EqualDev_Walk_empl,MI_MB_EqualDev_Walk_LC,MI_MB_EqualDev_Walk_GC,MI_MB_EqualDev_Walk_TC,MI_MB_EqualDev_Walk_Schools,MI_MB_Greenfield_Walk_empl,MI_MB_Greenfield_Walk_LC,MI_MB_Greenfield_Walk_GC,MI_MB_Greenfield_Walk_TC,MI_MB_Greenfield_Walk_Schools,MI_MB_mdINIS_Walk_empl,MI_MB_mdINIS_Walk_LC,MI_MB_mdINIS_Walk_GC,MI_MB_mdINIS_Walk_TC,MI_MB_mdINIS_Walk_Schools,MI_MB_TODInfill_Walk_empl,MI_MB_TODInfill_Walk_LC,MI_MB_TODInfill_Walk_GC,MI_MB_TODInfill_Walk_TC,MI_MB_TODInfill_Walk_Schools,MI_MB_Actual_PT_Combined,MI_MB_EqualDev_PT_Combined,MI_MB_Greenfield_PT_Combined,MI_MB_mdINIS_PT_Combined,MI_MB_TODInfill_PT_Combined,MI_MB_Actual_Cycle_Combined,MI_MB_EqualDev_Cycle_Combined,MI_MB_Greenfield_Cycle_Combined,MI_MB_mdINIS_Cycle_Combined,MI_MB_TODInfill_Cycle_Combined,MI_MB_Actual_Walk_Combined,MI_MB_EqualDev_Walk_Combined,MI_MB_Greenfield_Walk_Combined,MI_MB_mdINIS_Walk_Combined,MI_MB_TODInfill_Walk_Combined
0,1,10636200000,105,2991.452991,0.998223,0.998223,0.998223,0.998936,0.998260,1.918131,1.785453,1.333934,1.334836,1.835951,5.211176,5.783418,4.592555,4.607091,4.993043,0.182282,0.182282,0.182282,0.182284,0.182283,1.122617,1.016298,0.210439,0.210451,0.369539,1.600937,1.832319,1.581862,1.582389,1.624227,0.178995,0.178995,0.178995,0.178995,0.178995,1.061917,...,1.235294,6.298256,1.237016,1.235294,1.250944,1.141176,5.546157,1.142767,1.141176,1.334889,1.0,1.0,1.001394,1.0,1.0,1.0,1.000000,1.001394,1.0,1.00000,1.0,1.753522,1.001394,1.0,1.026612,1.757759,1.902105,0.276833,0.883463,0.292463,1.026900,1.052862,0.375676,0.907727,0.342453,0.965527,1.123479,0.227507,0.889797,0.253505
1,2,10636210000,122,2541.666667,1.072899,1.072899,1.072899,1.074156,1.072975,1.982997,1.828655,1.391537,1.393076,1.949945,5.361751,5.866793,4.848402,4.873082,5.312768,0.271061,0.271061,0.271061,0.271062,0.271062,0.977402,1.098533,0.280052,0.280063,0.737542,1.953577,2.072464,1.948012,1.948540,2.066809,0.307273,0.307273,0.307273,0.307273,0.307273,1.048405,...,1.051724,3.464120,1.052941,1.051724,1.054870,1.155172,4.095880,1.156509,1.155172,1.229452,1.0,1.0,1.001157,1.0,1.0,1.0,1.000000,1.001157,1.0,1.00000,1.0,2.525380,1.001157,1.0,1.067570,1.196653,1.695629,0.286789,0.889308,0.330958,0.862709,1.043195,0.382928,0.909728,0.344833,0.725693,1.008766,0.239191,0.896675,0.279775
2,3,10636220000,136,2839.248434,0.977449,0.977449,0.977449,0.978732,0.977527,1.911184,1.741602,1.260963,1.262533,1.875576,4.937069,5.347028,4.559510,4.584680,5.075894,0.286684,0.286684,0.286684,0.286686,0.286685,1.039740,0.692534,0.288624,0.288637,1.070592,1.861949,1.878522,1.861311,1.861839,2.236127,0.283587,0.283587,0.283587,0.283587,0.283587,1.034004,...,1.022556,3.705966,1.021335,1.022556,1.022792,1.127820,2.724936,1.126472,1.127820,1.139604,1.0,1.0,0.998805,1.0,1.0,1.0,1.000000,0.998805,1.0,1.00000,1.0,3.726957,0.998805,1.0,1.204762,1.193469,1.327620,0.298943,0.893865,0.395135,0.853197,1.026694,0.388679,0.909570,0.354209,0.723193,0.859900,0.249600,0.897581,

In [30]:
 MB_data = GeoAccessor.from_featureclass("MB21_CQ_pop_DSall")

In [60]:
round(15900*100/266100, 2)

5.98

In [64]:
# get all data from a particular SA3
SA3_list = MB_data["SA3_NAME21"].unique()

service = "Combined" # "Combined"
mode = "Walk"

for SA3 in SA3_list:
    PA_sum_CQ = pop_acc_df["PA_2016_"+ mode+"_" + service].sum()
    
    PA_sum_SA3 = pop_acc_df["PA_2016_"+ mode+"_"  + service][MB_data["SA3_NAME21"] == SA3].sum()
    
    percent_mode_est = round(PA_sum_SA3/PA_sum_CQ *100,2)
    
    print(SA3,":", percent_mode_est, "%")
    
    
# actual mode share distributions for comparison (Cycling)

# Queanbeyan : 0 %
# North Canberra : 30.25 %
# South Canberra : 9.77 %
# Gungahlin : 7.68 %
# Belconnen : 27.27 %
# Tuggeranong : 8.48 %
# Weston Creek/Molonglo : 11.85 %
# Woden Valley : 3.97 %

#limitation - amount of data is low, and so results could be easily distorted

#walking actual mode share distributions

# Queanbeyan : 8.49 %
# North Canberra : 15.78 %
# South Canberra : 13.64 %
# Gungahlin : 12.25 %
# Belconnen :  22.85 %
# Tuggeranong : 14.43 %
# Weston Creek/Molonglo : 6.58 %
# Woden Valley : 5.98 %

Queanbeyan : 5.91 %
North Canberra : 15.84 %
South Canberra : 7.8 %
Gungahlin : 16.59 %
Belconnen : 21.01 %
Tuggeranong : 17.92 %
Weston Creek : 5.16 %
Woden Valley : 8.99 %
Canberra East : 0.05 %
Uriarra - Namadgi : 0.0 %
Molonglo : 0.72 %


In [27]:
[print(col) for col in pop_acc_df.columns]

OBJECTID
MB_CODE21
Person
Population_density
Cycle_GC_Actual_Access
Cycle_GC_EqualDev_Access
Cycle_GC_Greenfield_Access
Cycle_GC_mdINIS_Access
Cycle_GC_TODInfill_Access
Cycle_LC_Actual_Access
Cycle_LC_EqualDev_Access
Cycle_LC_Greenfield_Access
Cycle_LC_mdINIS_Access
Cycle_LC_TODInfill_Access
Cycle_Schools_Actual_Access
Cycle_Schools_EqualDev_Access
Cycle_Schools_Greenfield_Access
Cycle_Schools_mdINIS_Access
Cycle_Schools_TODInfill_Access
PT_GC_Actual_Access
PT_GC_EqualDev_Access
PT_GC_Greenfield_Access
PT_GC_mdINIS_Access
PT_GC_TODInfill_Access
PT_LC_Actual_Access
PT_LC_EqualDev_Access
PT_LC_Greenfield_Access
PT_LC_mdINIS_Access
PT_LC_TODInfill_Access
PT_Schools_Actual_Access
PT_Schools_EqualDev_Access
PT_Schools_Greenfield_Access
PT_Schools_mdINIS_Access
PT_Schools_TODInfill_Access
Walk_GC_Actual_Access
Walk_GC_EqualDev_Access
Walk_GC_Greenfield_Access
Walk_GC_mdINIS_Access
Walk_GC_TODInfill_Access
Walk_LC_Actual_Access
Walk_LC_EqualDev_Access
Walk_LC_Greenfield_Access
Walk_LC_mdINIS_

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

### Export the layer
At this point we have a layer with the Accessibility, Population accessibility and mode share increase for each mode, service and development scenario, and a combined accessibility, population accesibility and mode share increase for each mode and development scenario.

In [86]:
MI_MB_df.spatial.to_featureclass(location = r"C:\Users\gjames\Documents\ArcGIS\Projects\Test_Visualisations\Test_Visualisations.gdb\MI_by_MB_sub_scratch")
#pop_acc_dens_df.spatial.to_featureclass(location = r"C:\Users\gjames\Documents\ArcGIS\Projects\Test_Visualisations\Test_Visualisations.gdb\PA_dens_DS21_scratch")

'C:\\Users\\gjames\\Documents\\ArcGIS\\Projects\\Test_Visualisations\\Test_Visualisations.gdb\\MI_by_MB_sub_scratch'

### Arrange in a format the dahsboard can use
Potential symbology issues with all accessibility values in a single layer - leave for another time. Potential convert to quantile (1-10) to keep consistent symbology.

#### Mode increases


Mode increases will be on the same scale, so can be compared directly. Aim for the format:

[MB, MI_dens, MI, Service, Scenario, SHAPE]

as the column names

In [29]:
## go through each mode, development scenario and service


mode_list = ["PT", "Cycle", "Walk"]

#note service list has added combined column
service_list = ["empl", "LC", "GC", "TC", "Schools", "Combined"]

DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

dashboard_dfs = {}

for mode in mode_list:
    
    #create a map for each mode, as modes can't be compared directly in a map - they can, but leave it for now
    dashboard_dfs[mode] = pd.DataFrame([])
    
    MB_list = []
    MI_dens_list = []
    MI_list = []
    Service_list = []
    Scenario_list = []
    Shape_list = []
    
    for DS in DS_list[1:]:
        for service in service_list:
            
            #get the columns for each of the output columns we want, and add them to the list
            
            for MB in pop_acc_df["MB_CODE21"]:
                MB_list.append(MB)
            
            for MI_dens in pop_acc_df["_".join(["MI_dens", DS, mode, service])]:
                MI_dens_list.append(MI_dens)
                
            for MI in pop_acc_df["_".join(["MI", DS, mode, service])]:
                MI_list.append(MI)
                
            for Shape in pop_acc_df["SHAPE"]:
                Shape_list.append(Shape)
                
                #add all the non-specific data too
                Service_list.append(service)
                Scenario_list.append(DS)
                
                
            
            
    dashboard_dfs[mode]["MB"] = MB_list
    dashboard_dfs[mode]["MI_dens"] = MI_dens_list
    dashboard_dfs[mode]["MI"] = MI_list
    dashboard_dfs[mode]["Service"] = Service_list
    dashboard_dfs[mode]["Scenario"] = Scenario_list
    dashboard_dfs[mode]["SHAPE"] = Shape_list
    
    
#     col_names = ["MB", "MI_dens", "MI", "Service", "Scenario", "SHAPE"]
#     dashboard_dfs[mode] = pd.DataFrame([[MB_list, MI_dens_list, MI_list, Service_list,Scenario_list,Shape_list]], columns = col_names)
    
dashboard_dfs["PT"]

,MB,MI_dens,MI,Service,Scenario,SHAPE
0,10636200000,0.002497,0.000088,empl,Actual,"{'rings': [[[149.21605164087055, -35.357119318..."
1,10636210000,0.003053,0.000147,empl,Actual,"{'rings': [[[149.21687245120268, -35.355530258..."
2,10636220000,0.003985,0.000191,empl,Actual,"{'rings': [[[149.21892156188062, -35.347781808..."
3,10636230000,0.003267,0.000107,empl,Actual,"{'rings': [[[149.22002280151116, -35.345635568..."
4,10636240000,0.005280,0.000117,empl,Actual,"{'rings': [[[149.22002280151116, -35.345635568..."
...,...,...,...,...,...,...
215545,85987000000,0.000000,0.000000,Combined,TODInfill,"{'rings': [[[149.14202945859267, -35.149113300..."
215546,85988000000,0.012592,0.000798,Combined,TODInfill,"{'rings': [[[149.1468375867853, -35.3127041626..."
215547,85989000000,0.000000,0.000000,Combined,TODInfill,"{'rings': [[[148.9828888458436, -35.2058493476..."
215548,85990000000,0.000000,0.000000,Combined,TODInfill,"{'rings': [[[148.9828888458436, -35.2058493476..."


In [30]:
#### Export the dashboard layer
#dashboard_dfs["PT"].spatial.to_featureclass(location = r"C:\Users\gjames\Documents\ArcGIS\Projects\Test_Visualisations\Test_Visualisations.gdb\MI_PT_dashboard_scratch")

Repeat for only combined services mode share increases (for faster rendering)

Also replace names for scenarios with more readable ones.

In [31]:
## go through each mode, development scenario and service


mode_list = ["PT", "Cycle", "Walk"]

#note service list has added combined column
service_list = ["Combined"]

DS_list = ["2016", "Actual", "EqualDev", "Greenfield", "mdINIS", "TODInfill"]

dashboard_dfs = {}

for mode in mode_list:
    
    #create a map for each mode, as modes can't be compared directly in a map - they can, but leave it for now
    dashboard_dfs[mode] = pd.DataFrame([])
    
    MB_list = []
    MI_dens_list = []
    MI_list = []
    Service_list = []
    Scenario_list = []
    Shape_list = []
    
    for DS in DS_list[1:]:
        
        # rename Development Scenarios to more readable names
        if DS == "EqualDev":
            DS_name = "Equal Development"
        elif DS == "mdINIS":
            DS_name = "Medium Density Inner Districts"
        elif DS == "TODInfill":
            DS_name = "TOD"
        else:
            DS_name = DS
        
        for service in service_list:
            
            #get the columns for each of the output columns we want, and add them to the list
            
            for MB in pop_acc_df["MB_CODE21"]:
                MB_list.append(MB)
            
            for MI_dens in pop_acc_df["_".join(["MI_dens", DS, mode, service])]:
                MI_dens_list.append(MI_dens)
                
            for MI in pop_acc_df["_".join(["MI", DS, mode, service])]:
                MI_list.append(MI)
                
            for Shape in pop_acc_df["SHAPE"]:
                Shape_list.append(Shape)
                
                #add all the non-specific data too
                Service_list.append(service)
                Scenario_list.append(DS_name)
                
                
            
            
    dashboard_dfs[mode]["MB"] = MB_list
    dashboard_dfs[mode]["MI_dens"] = MI_dens_list
    dashboard_dfs[mode]["MI"] = MI_list
    dashboard_dfs[mode]["Service"] = Service_list
    dashboard_dfs[mode]["Scenario"] = Scenario_list
    dashboard_dfs[mode]["SHAPE"] = Shape_list
    
    
#     col_names = ["MB", "MI_dens", "MI", "Service", "Scenario", "SHAPE"]
#     dashboard_dfs[mode] = pd.DataFrame([[MB_list, MI_dens_list, MI_list, Service_list,Scenario_list,Shape_list]], columns = col_names)
    
dashboard_dfs["PT"]

,MB,MI_dens,MI,Service,Scenario,SHAPE
0,10636200000,0.004517,0.000159,Combined,Actual,"{'rings': [[[149.21605164087055, -35.357119318..."
1,10636210000,0.004336,0.000208,Combined,Actual,"{'rings': [[[149.21687245120268, -35.355530258..."
2,10636220000,0.005136,0.000246,Combined,Actual,"{'rings': [[[149.21892156188062, -35.347781808..."
3,10636230000,0.003784,0.000124,Combined,Actual,"{'rings': [[[149.22002280151116, -35.345635568..."
4,10636240000,0.006511,0.000145,Combined,Actual,"{'rings': [[[149.22002280151116, -35.345635568..."
...,...,...,...,...,...,...
35920,85987000000,0.000000,0.000000,Combined,TOD,"{'rings': [[[149.14202945859267, -35.149113300..."
35921,85988000000,0.012592,0.000798,Combined,TOD,"{'rings': [[[149.1468375867853, -35.3127041626..."
35922,85989000000,0.000000,0.000000,Combined,TOD,"{'rings': [[[148.9828888458436, -35.2058493476..."
35923,85990000000,0.000000,0.000000,Combined,TOD,"{'rings': [[[148.9828888458436, -35.2058493476..."


In [33]:
## export all daashboard layers
# for mode in dashboard_dfs.keys():
#     layer_name = "MI_" + mode + "_dashboard_Combined"
#     dashboard_dfs[mode].spatial.to_featureclass(location = "C:/Users/gjames/Documents/ArcGIS/Projects/Test_Visualisations/Test_Visualisations.gdb/" + layer_name)
    
# dashboard_dfs[mode].spatial.to_featureclass(location = r"C:\Users\gjames\Documents\ArcGIS\Projects\Test_Visualisations\Test_Visualisations.gdb\MI_dashboard_scratch")
    

In [36]:
#export results tables
# overall_MI_df.to_csv(r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\Results\Combined_Mode_Increase.csv")
# MI_df.to_csv(r"C:\Users\gjames\OneDrive - Esri Australia\Honours\Data\Results\Mode_Increase_by_service.csv")